<a href="https://colab.research.google.com/github/RockhoRockho/Data-project/blob/main/22_intent_clsf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Processing 
* Intent Classification 테스크를 위한 데이터 처리

In [ ]:
pip install torch

In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.5 MB/s 


In [ ]:
pip install gensim

In [ ]:
pip install src

  Using cached src-0.0.7.zip (6.3 kB)
  ERROR: Failed building wheel for src
  Running setup.py clean for src
Failed to build src
    Running setup.py install for src ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-j3nxk4p4/src_211e5547103f4ed08da5105c3f06cab9/setup.py'"'"'; __file__='"'"'/tmp/pip-install-j3nxk4p4/src_211e5547103f4ed08da5105c3f06cab9/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-1kn5yldj/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/src Check the logs for full command output.


In [ ]:
pip install git+https://github.com/kmkurn/pytorch-crf

  Cloning https://github.com/kmkurn/pytorch-crf to /tmp/pip-req-build-glgj02qr
  Running command git clone -q https://github.com/kmkurn/pytorch-crf /tmp/pip-req-build-glgj02qr
  Created wheel for pytorch-crf: filename=pytorch_crf-0.7.2-py3-none-any.whl size=6289 sha256=1ff1ed8ff61b5e8d56d509047c603c070a235190100f1353c5b3cc18a34d7ac7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tgmwtyiy/wheels/5a/09/7f/a56999c3ea15a3adf587c4e900b4fd03c3c8a3cd49ad5b7dac
Successfully built pytorch-crf


In [ ]:
import os
import sys
import json
import torch
import random

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from tqdm import trange

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

from src.dataset import Preprocessing
from src.model import EpochLogger, MakeEmbed

ModuleNotFoundError: ignored

In [ ]:
class MakeDataset:
    def __init__(self):
        
        self.intent_label_dir = "./data/dataset/intent_label.json"
        self.intent_data_dir = "./data/dataset/intent_data.csv"
        
        self.intent_label = self.load_intent_label()
        self.prep = Preprocessing()

    def load_intent_label(self):
        f = open(self.intent_label_dir, encoding="UTF-8") 
        intent_label = json.loads(f.read())
        self.intents = list(intent_label.keys())
        return intent_label
    
    def tokenize(self, sentence):
        return sentence.split()

    def tokenize_dataset(self, dataset):
        token_dataset = []
        for data in dataset:
            token_dataset.append(self.tokenize(data))
        return token_dataset

    def make_intent_dataset(self, embed): # intent 분류를 위한 Dataset 생성
        intent_dataset = pd.read_csv(self.intent_data_dir) # 데이터를 로드

        labels = [self.intent_label[label] for label in intent_dataset["label"].to_list()] # label
        
        # 사용자 발화 : 사용자가 챗봇과 커뮤니케이션하기 위해 내뱉는 말 또는 텍스트등을 의미
        intent_querys = self.tokenize_dataset(intent_dataset["question"].tolist()) # 사용자 발화 tokenize
        
        dataset = list(zip(intent_querys, labels)) # (사용자 발화, intent) 형태로 가공
        intent_train_dataset, intent_test_dataset = self.word2idx_dataset(dataset, embed) # word2index
        return intent_train_dataset, intent_test_dataset

    def word2idx_dataset(self, dataset ,embed, train_ratio = 0.8):
        embed_dataset = []
        question_list, label_list = [], []
        flag = True
        random.shuffle(dataset) # 훈련용과 검증용으올 나눌 때 intent 편형이 나타나지 않도록 데이터를 셔플
        for query, label in dataset :
            q_vec = embed.query2idx(query) # 사용자 발화  index 형태로 변환
            q_vec = self.prep.pad_idx_sequencing(q_vec) # 사용자 발화 최대길이 까지 padding

            question_list.append(torch.tensor([q_vec]))
            label_list.append(torch.tensor([label]))

        x = torch.cat(question_list)
        y = torch.cat(label_list)

        # 학습용과 검증용으로 나누기
        x_len = x.size()[0]
        y_len = y.size()[0]
        if(x_len == y_len):
            train_size = int(x_len*train_ratio)
            
            
            train_x = x[:train_size]
            train_y = y[:train_size]

            test_x = x[train_size+1:]
            test_y = y[train_size+1:]
            
            # TensorDataset으로 감싸기
            # 파이토치의 TensorDataset은 tensor를 감싸는 Dataset
            # 인덱싱 방식과 길이를 정의함으로써 tensor의 첫 번째 차원을 따라 반복, 인덱스, 슬라이스를 위한 방법을 제공하게 됨
            # 학습할 때 동일한 라인에서 독립 변수와 종속 변수에 쉽게 접근할 수 있음
            train_dataset = TensorDataset(train_x,train_y)
            test_dataset = TensorDataset(test_x,test_y)
            
            return train_dataset, test_dataset
            
        else:
            print("ERROR x!=y")


# Convolutional Neural 
### tensorflow code : https://github.com/SeonbeomKim/TensorFlow-TextCNN/blob/master/TextCNN.py


# nn.Conv2d(in_channels, out_channels, kernel_size)


In [ ]:
class textCNN(nn.Module):
    
    def __init__(self, w2v, dim, kernels, dropout, num_class):
        super(textCNN, self).__init__()
        vocab_size = w2v.size()[0]
        emb_dim = w2v.size()[1]
        self.embed = nn.Embedding(vocab_size+2, emb_dim)
        self.embed.weight[2:].data.copy_(w2v)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim, (w, emb_dim)) for w in kernels])
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(len(kernels)*dim, num_class)

    def forward(self, x):
            emb_x = self.embed(x)
            emb_x = emb_x.unsqueeze(1)

            con_x = [conv(emb_x) for conv in self.convs]
            # [(out_channels, conv결과 길이), ...]

            pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x] 
            # [(256, 1), ...]

            fc_x = torch.cat(pool_x, dim=1) # concat 하여 fcl Layer의 입력 형태로 만듬

            fc_x = fc_x.squeeze(-1) # 차원 맞추기
            fc_x = self.dropout(fc_x)
            logit = self.fc(fc_x)
            return logit

# 모델의 가중치 저장을 위한 코드
def save(model, save_dir, save_prefix, epoch):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, save_prefix)
    save_path = '{}_steps_{}.pt'.format(save_prefix, epoch)
    torch.save(model.state_dict(), save_path)


In [ ]:
weights = embed.word2vec.wv.vectors
weights = torch.FloatTensor(weights)

num_class = len(dataset.intent_label) 
model = textCNN(weights, 256, [3,4,5], 0.5, num_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

NameError: ignored

In [ ]:
model

In [ ]:
%%time
epoch = 10
prev_acc = 0
save_dir = "./data/pretraining/1_intent_clsf_model/"
save_prefix = "intent_clsf"
for i in range(epoch):
    steps = 0
    model.train() 
    #for data in train_dataloader:
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]
            logit = model.forward(x)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(logit, target) # loass function
            loss.backward()
            optimizer.step()

            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            tepoch.set_postfix(loss=loss.item(), accuracy= accuracy.numpy())

    model.eval() # weight 업데이트 금지
    steps = 0
    accuarcy_list = []
    #for data in test_dataloader:
    with tqdm(test_dataloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {i}")
            x = data[0]
            target = data[1]

            logit = model.forward(x)
            loss = F.cross_entropy(logit, target)
            corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
            accuracy = 100.0 * corrects/x.size()[0]
            accuarcy_list.append(accuracy.tolist())
            
            tepoch.set_postfix(loss=loss.item(), accuracy= sum(accuarcy_list)/len(accuarcy_list))
            
    acc = sum(accuarcy_list)/len(accuarcy_list)
    if(acc>prev_acc):
        prev_acc = acc
        save(model, save_dir, save_prefix+"_"+str(round(acc,3)), i)


# Load & Test

In [ ]:
model.load_state_dict(torch.load("./data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"))

model.eval()

In [ ]:
%%time
q = "제주도 오늘 날씨 알려줘"

x = dataset.prep.pad_idx_sequencing(embed.query2idx(dataset.tokenize(q)))

x = torch.tensor(x)
f = model(x.unsqueeze(0))

intent = dataset.intents[torch.argmax(f).tolist()]

print("발화 : " + q)
print("의도 : " + intent)
